# Analyse year-on-year variability of annual energy demand

In [ ]:
# Hack to emulate running notebook from root directory.
import os
os.chdir('..')

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_dir = os.path.join('data','from-database')
bname_pattern = 'UCam_Building_b%s.csv'
outname_pattern = 'ly_{building_id}-{year}.csv'

years = list(range(2012, 2018))
building_ids = [0, 4, 8, 19, 25, 40, 58, 102, 104] # 118

In [ ]:
hours_per_year = 24*365
timestamps = pd.read_csv(os.path.join(data_dir, 'timestamps.csv'))
timestamps['Timestamp (UTC)'] = pd.to_datetime(timestamps['Timestamp (UTC)'])

In [ ]:
elec_ann_loads = {}
heat_ann_loads = {}

for b_id in building_ids:
    load_data = pd.read_csv(os.path.join(data_dir, bname_pattern % b_id), header=0)

    elec_ann_loads[b_id] = []
    heat_ann_loads[b_id] = []

    for year in years:
        year_first_idx = timestamps.index[timestamps['Timestamp (UTC)'].dt.year == year].min()

        elec_load = load_data.loc[year_first_idx:year_first_idx+hours_per_year-1, 'Equipment Electric Power [kWh]'].to_numpy()
        heat_load = load_data.loc[year_first_idx:year_first_idx+hours_per_year-1, 'Heating Load [kWh]'].to_numpy()

        elec_ann_loads[b_id].append(np.sum(elec_load))
        heat_ann_loads[b_id].append(np.sum(heat_load))

In [ ]:
norm_elec_ann_loads = {k: np.array(v)/np.mean(v) for k,v in elec_ann_loads.items()}
norm_heat_ann_loads = {k: np.array(v)/np.mean(v) for k,v in heat_ann_loads.items()}

In [ ]:
print(norm_elec_ann_loads)
print(norm_heat_ann_loads)

In [ ]:
all_norm_elec_ann_loads = np.array([v for k,v in norm_elec_ann_loads.items()]).flatten()
all_norm_heat_ann_loads = np.array([v for k,v in norm_heat_ann_loads.items()]).flatten()

In [ ]:
print(np.std(all_norm_elec_ann_loads))
print(np.std(all_norm_heat_ann_loads))

In [ ]:
fig, ax = plt.subplots()
sns.kdeplot(all_norm_elec_ann_loads, ax=ax, c='k', cut=0, levels=200, label='Electricity')
sns.kdeplot(all_norm_heat_ann_loads, ax=ax, c='b', cut=0, levels=200, label='Heat')
plt.xlabel("Load variation")
plt.ylabel("Density")
ax.set_yticks([])
ax.set_yticklabels([])
plt.legend()
#plt.savefig(os.path.join('plots',"..."), format="pdf", bbox_inches="tight")
plt.show()

Note this variation is over a 6 year period, so it will likely be greater over the 20 year assumed operational lifetime.